# Initialize

In [42]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
import $ivy.`sh.almond::almond-spark:0.9.1` // Not required since almond 0.7.0 (will be automatically added when importing spark)

import $ivy.$                                   // Or use any other 2.x version here

import org.apache.log4j.{Level, Logger}

import $ivy.$                               // Not required since almond 0.7.0 (will be automatically added when importing spark)

# Create Spark session

In [43]:
import org.apache.spark.sql._
val sparkSession = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
sparkSession.getClass
import sparkSession.sql
import sparkSession.implicit._

Getting spark JARs
Creating SparkSession


Spark UI

import org.apache.spark.sql._

sparkSession: SparkSession = org.apache.spark.sql.SparkSession@1541203
res42_2: Class[T] = class org.apache.spark.sql.SparkSession
import sparkSession.sql

# Load data

In [61]:
val result = sparkSession.read.format("csv")
      .option("delimiter", "\t")
      .option("header", "true")
      .option("quote", "'")
      .option("nullValue", "NULL")
      .option("inferSchema", "true")
      .load("/home/pi/sparktest/data/privpref")

load at cmd60.sc:7

1 / 1

load at cmd60.sc:7

1 / 1

load at cmd60.sc:7

4 / 4

load at cmd60.sc:7

4 / 4

result: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]

In [62]:
result.getClass
result.createOrReplaceTempView("result")
result.printSchema
result.first.getClass

root
 |-- REF_NUM0: string (nullable = true)
 |-- LAST_USED_DT: string (nullable = true)
 |-- LAST_USED_DT_OLD: string (nullable = true)
 |-- SOURCE_IDENT_TP_CD: string (nullable = true)
 |-- NAME4: string (nullable = true)
 |-- X_COUNTRY_TP_CD: string (nullable = true)
 |-- ISO_CODE: string (nullable = true)
 |-- NAME7: string (nullable = true)
 |-- CONT_ID: string (nullable = true)
 |-- CONT_METH_TP_CD: string (nullable = true)
 |-- REF_NUM10: string (nullable = true)
 |-- MINSTNUM: string (nullable = true)
 |-- CNT: string (nullable = true)
 |-- MAXPP: string (nullable = true)
 |-- MINPP: string (nullable = true)
 |-- LOCATION_GROUP_ID: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)



first at cmd61.sc:4

1 / 1

first at cmd61.sc:4

1 / 1

res61_0: Class[T] = class org.apache.spark.sql.Dataset
res61_3: Class[T] = class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema

# get records with different SP

In [63]:
val differentSP = sql("""
select * from result 
where 
((minPP = 'S' and maxPP = 'P') or (minPP = 'P' and maxPP = 'S'))
""")
differentSP.cache
differentSP.createOrReplaceTempView("differentSP")
differentSP.show(5)

show at cmd62.sc:8

1 / 1

show at cmd62.sc:8

1 / 1

+---------------+--------------------+--------------------+------------------+-----+---------------+--------+-----+------------------+---------------+--------------+--------------+---+-----+-----+------------------+----+-----+---+
|       REF_NUM0|        LAST_USED_DT|    LAST_USED_DT_OLD|SOURCE_IDENT_TP_CD|NAME4|X_COUNTRY_TP_CD|ISO_CODE|NAME7|           CONT_ID|CONT_METH_TP_CD|     REF_NUM10|      MINSTNUM|CNT|MAXPP|MINPP| LOCATION_GROUP_ID|YEAR|MONTH|DAY|
+---------------+--------------------+--------------------+------------------+-----+---------------+--------+-----+------------------+---------------+--------------+--------------+---+-----+-----+------------------+----+-----+---+
|630881052626909|2020-07-22 20:27:...|2020-07-21 20:27:...|            100005|  IWM|            185|      US| WORK|743404802239030641|              2|    7142503000|  +17142503000|  2|    S|    P|720046741219991462|2020|    8| 10|
|161521056084933|2020-07-31 12:55:...|2020-07-30 12:55:...|            10000

differentSP: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
res62_1: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]

# get records with: one is s/p, other is N

In [64]:
val spn = sql("""
select * from result where 
        ((minPP = 'S' or minPP = 'P') and maxPP = 'N')
        or
        ((maxPP = 'S' or maxPP = 'P') and minPP = 'N')
""")
spn.cache
spn.createOrReplaceTempView("spn")
spn.show(5)

show at cmd63.sc:9

1 / 1

show at cmd63.sc:9

1 / 1

+---------------+--------------------+--------------------+------------------+-------+---------------+--------+-----+------------------+---------------+------------+-------------+---+-----+-----+------------------+----+-----+---+
|       REF_NUM0|        LAST_USED_DT|    LAST_USED_DT_OLD|SOURCE_IDENT_TP_CD|  NAME4|X_COUNTRY_TP_CD|ISO_CODE|NAME7|           CONT_ID|CONT_METH_TP_CD|   REF_NUM10|     MINSTNUM|CNT|MAXPP|MINPP| LOCATION_GROUP_ID|YEAR|MONTH|DAY|
+---------------+--------------------+--------------------+------------------+-------+---------------+--------+-----+------------------+---------------+------------+-------------+---+-----+-----+------------------+----+-----+---+
|190771087546590|2020-08-06 05:04:...|2020-08-05 05:04:...|            100010|DSW SAP|             90|      JP| WORK|855438468796951971|              2|079-295-4111| +81792954111|  2|    P|    N|214854384688367691|2020|    8| 10|
|454901185102288|2020-08-04 20:28:...|2020-08-03 20:28:...|            100011|DS

spn: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
res63_1: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]

# get only one has s/p

In [65]:
val onlyone = sql("""
select * from result where 
        ((maxpp = 'S' or maxpp = 'P')and minpp is null)
            or
        ((minpp = 'S' or minpp = 'P')and maxpp is null)
""")
onlyone.cache
onlyone.createOrReplaceTempView("onlyone")
onlyone.count

count at cmd64.sc:9

4 / 4

count at cmd64.sc:9

4 / 4

count at cmd64.sc:9

1 / 1

count at cmd64.sc:9

1 / 1

onlyone: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
res64_1: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
res64_3: Long = 438L

# other queries

In [70]:
val onlyone10 = sql("""
select * from onlyone
where day = 10
""")
onlyone10.cache
onlyone10.createOrReplaceTempView("onlyone10")
print("onlyone on day 10: " + onlyone10.count)

val onlyone7 = sql("""
select * from onlyone
where day = 7
""")
onlyone7.cache
onlyone7.createOrReplaceTempView("onlyone7")
print("onlyone on day 7: " + onlyone7.count)

val sp10 = sql ("""
select * from differentsp where day = 10
""")
print("different sp on day 10: " + sp10.count)

val sp7 = sql("""
select * from differentsp where day = 7
""")
print("different sp on day 7: " + sp7.count)

val spn10 = sql ("""
select * from spn where day = 10
""")
print("sp and n on day 10: " + spn10.count)

val spn7 = sql("""
select * from spn where day = 7
""")
print("sp and n on day 7: " + spn7.count)



count at cmd69.sc:7

4 / 4

count at cmd69.sc:7

4 / 4

count at cmd69.sc:7

...

1 / 1

count at cmd69.sc:35

1 / 1

sp and n on day 7: 8

onlyone10: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
res69_1: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
onlyone7: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
res69_5: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
sp10: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
sp7: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
spn10: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]
spn7: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]

In [71]:
sql("""
select * from differentsp where day = 3
""").show(3)

show at cmd70.sc:3

1 / 1

show at cmd70.sc:3

1 / 1

show at cmd70.sc:3

3 / 3

show at cmd70.sc:3

3 / 3

+---------------+--------------------+--------------------+------------------+-----+---------------+--------+-----+------------------+---------------+------------+------------+---+-----+-----+------------------+----+-----+---+
|       REF_NUM0|        LAST_USED_DT|    LAST_USED_DT_OLD|SOURCE_IDENT_TP_CD|NAME4|X_COUNTRY_TP_CD|ISO_CODE|NAME7|           CONT_ID|CONT_METH_TP_CD|   REF_NUM10|    MINSTNUM|CNT|MAXPP|MINPP| LOCATION_GROUP_ID|YEAR|MONTH|DAY|
+---------------+--------------------+--------------------+------------------+-----+---------------+--------+-----+------------------+---------------+------------+------------+---+-----+-----+------------------+----+-----+---+
|630881052626909|2020-07-22 20:27:...|2020-07-21 20:27:...|            100005|  IWM|            185|      US| WORK|743404802239030641|              2|  7142503000|+17142503000|  2|    S|    P|720046741219991462|2020|    8|  3|
|161521056084933|2020-07-31 12:55:...|2020-07-30 12:55:...|            100005|  IWM|        

## find data existed in two consecutive days

In [50]:
sql("""
select count(*) from onlyone5 t1 inner join onlyone7 t2
on t1.cont_id = t2.cont_id
""").show

run at ThreadPoolExecutor.java:1149

3 / 3

run at ThreadPoolExecutor.java:1149

3 / 3

show at cmd49.sc:1

...

show at cmd49.sc:1

1 / 1

show at cmd49.sc:1

1 / 1

+--------+
|count(1)|
+--------+
|      51|
+--------+



## find data only existed in next day

In [75]:
//use temp table:
sql("""
select t1.cont_id as count_id_5,t2.cont_id as count_id_7 from onlyone7 t1 right outer join onlyone10 t2
on t1.cont_id = t2.cont_id
where t1.cont_id is null
""").count

run at ThreadPoolExecutor.java:1149

4 / 4

run at ThreadPoolExecutor.java:1149

4 / 4

count at cmd74.sc:1

...

count at cmd74.sc:1

1 / 1

count at cmd74.sc:1

1 / 1

res74: Long = 16L

In [77]:
//use total table and filter
val onlyin10 = sql("""
select t1.cont_id as cont_id7,t2.cont_id as cont_id10, length(t2.cont_id) from 
(
    select * from onlyone where day = 7
) t1
right outer join 
(
    select * from onlyone where day = 10
) t2
on t1.cont_id = t2.cont_id
where t1.cont_id is null
""")
onlyin10.cache
onlyin10.show

show at cmd76.sc:14

1 / 1

show at cmd76.sc:14

1 / 1

show at cmd76.sc:14

...

1 / 1

show at cmd76.sc:14

1 / 1

+--------+------------------+---------------+
|cont_id7|         cont_id10|length(cont_id)|
+--------+------------------+---------------+
|    null|534454667583867712|             18|
|    null|731342367823494592|             18|
|    null|180048186274185672|             18|
|    null|524942415429393122|             18|
|    null|396242401815334131|             18|
|    null|999749920919879841|             18|
|    null|998640563435571692|             18|
|    null|899051567956550702|             18|
|    null|494042428996323822|             18|
|    null|190142411820879081|             18|
|    null|535052712246115282|             18|
|    null|653442393357741731|             18|
|    null|236653740158044062|             18|
|    null|651546643056147612|             18|
|    null|482557368669675302|             18|
|    null|196859009898864902|             18|
+--------+------------------+---------------+



onlyin10: DataFrame = [cont_id7: string, cont_id10: string ... 1 more field]
res76_1: DataFrame = [cont_id7: string, cont_id10: string ... 1 more field]

## Save result as CSV

In [73]:
val onlyone10 = sql("""
select * from onlyone where day = 10
""")
onlyone10.printSchema

root
 |-- REF_NUM0: string (nullable = true)
 |-- LAST_USED_DT: string (nullable = true)
 |-- LAST_USED_DT_OLD: string (nullable = true)
 |-- SOURCE_IDENT_TP_CD: string (nullable = true)
 |-- NAME4: string (nullable = true)
 |-- X_COUNTRY_TP_CD: string (nullable = true)
 |-- ISO_CODE: string (nullable = true)
 |-- NAME7: string (nullable = true)
 |-- CONT_ID: string (nullable = true)
 |-- CONT_METH_TP_CD: string (nullable = true)
 |-- REF_NUM10: string (nullable = true)
 |-- MINSTNUM: string (nullable = true)
 |-- CNT: string (nullable = true)
 |-- MAXPP: string (nullable = true)
 |-- MINPP: string (nullable = true)
 |-- LOCATION_GROUP_ID: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)



onlyone10: DataFrame = [REF_NUM0: string, LAST_USED_DT: string ... 17 more fields]

In [74]:
//file will created at /home/pi/jupyterNotebook/onlyin7.csv folder
//onlyin7.rdd.saveAsTextFile("onlyin7.csv")

val onlyone10 = sql("""
select 
REF_NUM0
, date_format(LAST_USED_DT,"yyyy-MM-dd HH:mm:ss.SSS")
, date_format(LAST_USED_DT_OLD,"yyyy-MM-dd HH:mm:ss.SSS")
,SOURCE_IDENT_TP_CD
,NAME4
,X_COUNTRY_TP_CD
,ISO_CODE
,NAME7
,CONT_ID
,CONT_METH_TP_CD
,REF_NUM10
,MINSTNUM
,CNT
,MAXPP
,MINPP
,LOCATION_GROUP_ID
,YEAR
,MONTH
,DAY 
from onlyone
where day = 10
""")
onlyone10.write
      .mode(SaveMode.Overwrite)
      .option("delimiter", "\t")
    .option("header",true)
     // .option("quote", "")
      .format("csv")
      .save("onlyone10")

save at cmd73.sc:31

4 / 4

save at cmd73.sc:31

4 / 4

save at cmd73.sc:62

...

save at cmd73.sc:93

4 / 4

save at cmd73.sc:93

4 / 4

onlyone10: DataFrame = [REF_NUM0: string, date_format(CAST(LAST_USED_DT AS TIMESTAMP), yyyy-MM-dd HH:mm:ss.SSS): string ... 17 more fields]
sp10: DataFrame = [REF_NUM0: string, date_format(CAST(LAST_USED_DT AS TIMESTAMP), yyyy-MM-dd HH:mm:ss.SSS): string ... 17 more fields]
spn10: DataFrame = [REF_NUM0: string, date_format(CAST(LAST_USED_DT AS TIMESTAMP), yyyy-MM-dd HH:mm:ss.SSS): string ... 17 more fields]

## find data only existed in previous day

In [25]:
sql("""
select t1.cont_id as count_id_5,t2.cont_id as count_id_7 from onlyone5 t1 left outer join onlyone7 t2
on t1.cont_id = t2.cont_id
where t2.cont_id is null
""").show

: 

In [40]:
import sparkSession.implicits._
result.select($"cont_id",$"cont_id"+1).show(5)

sql("""
select cont_id,cont_id + 1 from result
""").show(5)

show at cmd39.sc:2

1 / 1

+------------------+------------------+
|           cont_id|     (cont_id + 1)|
+------------------+------------------+
|743404802239030641|743404802239030642|
|950942431017746332|950942431017746333|
|500840484788116941|500840484788116942|
|453246376116782062|453246376116782063|
|481440522433109952|481440522433109953|
+------------------+------------------+
only showing top 5 rows



show at cmd39.sc:6

1 / 1

+------------------+-----------------------------+
|           cont_id|(cont_id + CAST(1 AS BIGINT))|
+------------------+-----------------------------+
|743404802239030641|           743404802239030642|
|950942431017746332|           950942431017746333|
|500840484788116941|           500840484788116942|
|453246376116782062|           453246376116782063|
|481440522433109952|           481440522433109953|
+------------------+-----------------------------+
only showing top 5 rows



import sparkSession.implicits._


In [79]:
import sparkSession.implicits._
case class Person(name:String, age:Long)


import sparkSession.implicits._

defined class Person

In [84]:
val personRDD = sparkSession.sparkContext.parallelize(Seq(Person("Andy",23)))
val personDF = sparkSession.createDataFrame(personRDD)

: 